# **CPU Energy Model Construction Notebook**

This notebook builds a server-specific energy model by analyzing stress experiment runs.  
The aim is to estimate the **per-core power consumption** based on controlled workloads, and use this to form a predictive model for energy estimation.

We assume:
- Each experiment run uses a known number of CPU cores.
- Power consumption is measured over time.
- The base (idle) power consumption of the system is known or measured separately.

In [ ]:
import os
import json
from datetime import datetime
from typing import Dict, List, Optional, Tuple

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import seaborn as sns

# === Constants ===
POWER_COLUMN = "power_active_W"
CPU_MODEL_FOLDER = "data/cpu_models"
GPU_MODEL_FOLDER = "data/gpu_models"

# === Build User Config ===
def build_config(server_name=None, result_subfolder=None, idle_power=None, model_kind=None):
    if not server_name:
        server_name = input("Enter the server name (e.g., riga): ").strip()

    if not result_subfolder:
        result_subfolder = input("Enter the result subfolder name (e.g., 2024_stress_4_cores): ").strip()

    if not model_kind:
        model_kind = input("Model type? [cpu/gpu] (default: cpu): ").strip().lower() or "cpu"
    if model_kind not in {"cpu", "gpu"}:
        raise ValueError("model_kind must be 'cpu' or 'gpu'")

    try:
        idle_power_input = input("Enter the idle power in watts (e.g., 58.0): ").strip()
        idle_power = float(idle_power_input)
    except ValueError:
        print("Invalid number. Please enter a valid float value (e.g., 58.0).")
        raise

    base_result_path = os.path.join("/srv/testbed/results/warmuth/default")
    full_result_folder = os.path.join(base_result_path, result_subfolder)
    if not os.path.exists(full_result_folder):
        raise ValueError(f"Warning: The folder {full_result_folder} does not exist.")

    # Defaults differ for cpu vs gpu
    if model_kind == "cpu":
        map_to = "cores"
        model_output = "cpu_model"
        x_col = "cores"
    else:
        map_to = "gpu_load"
        model_output = "gpu_model"
        x_col = "gpu_load"

    return {
        "result_folder": str(full_result_folder),
        "server_name": server_name,
        "idle_power": idle_power,
        "csv_base_name": "measurement_run",
        "csv_extension": ".csv",
        "map_to": map_to,
        "model_output": model_output,
        "model_kind": model_kind,
        "x_col": x_col,
        "fix_idle": False,

        # GPU defaults: 25/50/75/100
        "gpu_load_levels": [0.25, 0.50, 0.75, 1.00],
    }

# === Configuration Utilities ===
def validate_config(config: Dict, required_keys: List[str]) -> None:
    missing = [key for key in required_keys if key not in config]
    if missing:
        raise ValueError(f"Missing required config keys: {missing}")

def get_csv_folder(config: Dict) -> str:
    return os.path.join(config["result_folder"], "energy", config["server_name"])

# === Data Loading and Processing ===
def generate_run_mapping_cpu(config: Dict, fallback_cores: int = 4) -> Dict[str, int]:
    if "start_index" not in config or "end_index" not in config:
        cpu_info = load_cpu_info(config)
        cores = cpu_info.get("cores", fallback_cores)
        config["start_index"] = 0
        config["end_index"] = cores - 1

    start = config["start_index"]
    end = config["end_index"]
    max_digits = len(str(end + 1))

    mapping = {
        f"{config['csv_base_name']}{str(i).zfill(max_digits)}{config['csv_extension']}": i + 1
        for i in range(start, end + 1)
    }
    print(f"Run mapping for '{config['map_to']}':\n{json.dumps(mapping, indent=2)}")
    return mapping


def generate_run_mapping_gpu(config: Dict) -> Dict[str, float]:
    # assumes runs 0..len(levels)-1 correspond to 25/50/75/100 in order
    levels = config.get("gpu_load_levels", [0.25, 0.50, 0.75, 1.00])
    max_digits = len(str(len(levels)))

    mapping = {
        f"{config['csv_base_name']}{str(i).zfill(max_digits)}{config['csv_extension']}": float(level)
        for i, level in enumerate(levels)
    }
    print(f"Run mapping for '{config['map_to']}':\n{json.dumps(mapping, indent=2)}")
    return mapping


def generate_run_mapping(config: Dict, fallback_cores: int = 4) -> Dict:
    if config.get("model_kind", "cpu") == "gpu":
        return generate_run_mapping_gpu(config)
    return generate_run_mapping_cpu(config, fallback_cores=fallback_cores)

def merge_energy_channels(df: pd.DataFrame) -> pd.DataFrame:
    """Merge _0/_1 columns into unified channel columns."""
    prefixes = {
        col.rsplit('_', 1)[0]
        for col in df.columns
        if col.endswith('_0') or col.endswith('_1')
    }
    for prefix in prefixes:
        col0, col1 = f"{prefix}_0", f"{prefix}_1"
        if col0 in df.columns and col1 in df.columns:
            df[prefix] = df[col0] + df[col1]
            df.drop(columns=[col0, col1], inplace=True)
    return df

def load_power_data(
    csv_folder: str,
    run_mapping: Dict[str, float],
    p_base: float,
    x_col: str,
    model_kind: str,
    power_column: str = POWER_COLUMN
) -> pd.DataFrame:
    results = []
    for filename, x_val in run_mapping.items():
        path = os.path.join(csv_folder, filename)
        df = pd.read_csv(path)
        df = merge_energy_channels(df)
        avg_power = float(df[power_column].mean())

        row = {
            "filename": filename,
            x_col: float(x_val),
            "avg_power": round(avg_power, 2),
        }

        # Optional: “per-unit” metric (kept for convenience / sanity checks)
        if model_kind == "cpu":
            cores = float(x_val)
            row["per_unit_power"] = round((avg_power - p_base) / cores, 2) if cores > 0 else np.nan
        else:
            load = float(x_val)
            row["per_unit_power"] = round((avg_power - p_base) / load, 2) if load > 0 else np.nan

        results.append(row)

    print(f"Loaded data for {len(results)} runs.")
    return pd.DataFrame(results)

# === Modeling ===
def fit_models(
    model_df: pd.DataFrame,
    p_base: float,
    x_col: str,
    fix_idle: bool = False
) -> Tuple[LinearRegression, LinearRegression, PolynomialFeatures]:
    X = model_df[[x_col]]
    y = model_df["avg_power"]

    if fix_idle:
        reg_linear = LinearRegression(fit_intercept=False).fit(X, y - p_base)
        model_df["predicted_linear"] = reg_linear.predict(X) + p_base
    else:
        reg_linear = LinearRegression().fit(X, y)
        model_df["predicted_linear"] = reg_linear.predict(X)

    model_df["error_linear"] = (
        (model_df["avg_power"] - model_df["predicted_linear"]).abs()
        / model_df["avg_power"] * 100
    )

    poly = PolynomialFeatures(degree=2)
    X_poly = poly.fit_transform(X)

    if fix_idle:
        reg_poly = LinearRegression(fit_intercept=False).fit(X_poly, y - p_base)
        model_df["predicted_poly"] = reg_poly.predict(X_poly) + p_base
    else:
        reg_poly = LinearRegression().fit(X_poly, y)
        model_df["predicted_poly"] = reg_poly.predict(X_poly)

    model_df["error_poly"] = (
        (model_df["avg_power"] - model_df["predicted_poly"]).abs()
        / model_df["avg_power"] * 100
    )

    print("Model fitting complete.")
    return reg_linear, reg_poly, poly

def predict_cpu_power_detailed(
    per_core_loads: List[float],
    per_core_powers: List[float],
    p_base: float
) -> float:
    return p_base + sum(l * p for l, p in zip(per_core_loads, per_core_powers))

# === Visualization ===
def plot_predicted_load_levels(
    num_cores: Optional[int] = None,
    p_core: Optional[float] = None,
    p_base: Optional[float] = None,
    load_levels: Optional[List[float]] = None,
    labels: Optional[List[str]] = None,
    cpu_model: Optional[Dict] = None,
    method: str = "linear",
    per_core_steps: bool = False
) -> None:
    """
    Plot predicted total server power at different CPU load levels
    or for each number of active cores.
    """
    if cpu_model:
        num_cores = cpu_model.get("cpu_info", {}).get("cores")

    if num_cores is None:
        raise ValueError("Missing 'num_cores' information.")

    if per_core_steps:
        core_counts = list(range(1, num_cores + 1))
        labels = [f"{cores} cores" for cores in core_counts]
        predicted = []
        for cores in core_counts:
            if cpu_model:
                predicted_power = predict_power(cpu_model, num_cores=cores, method=method)
            else:
                predicted_power = p_base + (p_core * cores)
            predicted.append(predicted_power)
    else:
        load_levels = load_levels or [0.0, 0.25, 0.5, 1.0]
        labels = labels or [f"{int(load * 100)}% Load" if load > 0 else "Idle" for load in load_levels]
        predicted = []
        for load in load_levels:
            active_cores = num_cores * load
            if cpu_model:
                if active_cores == 0:
                    predicted_power = cpu_model["linear_model"]["p_base"]
                else:
                    predicted_power = predict_power(cpu_model, num_cores=active_cores, method=method)
            else:
                predicted_power = p_base + (p_core * active_cores)
            predicted.append(predicted_power)

    plt.figure(figsize=(8, 5))
    bars = plt.bar(labels, predicted, color="skyblue", edgecolor="black")
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 1, f"{yval:.1f} W",
                 ha='center', va='bottom', fontsize=10)
    title_mode = "Per Core Steps @100% Load" if per_core_steps else f"{method.capitalize()} Model"
    plt.title(f"Predicted Server Power ({title_mode})")
    plt.ylabel("Power (W)")
    plt.ylim(0, max(predicted) + 20)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()

def plot_model_fit(
    model_df: pd.DataFrame,
    reg_linear: LinearRegression,
    reg_poly: LinearRegression,
    poly: PolynomialFeatures,
    x_col: str = "cores"
) -> None:
    x_min = model_df[x_col].min()
    x_max = model_df[x_col].max()

    x_vals = pd.DataFrame({
        x_col: np.linspace(x_min, x_max, 100)
    })

    y_linear = reg_linear.predict(x_vals)
    y_poly = reg_poly.predict(poly.transform(x_vals))

    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(8, 5))
    sns.scatterplot(data=model_df, x=x_col, y="avg_power", s=80, label="Measured")
    plt.plot(x_vals[x_col], y_linear, color="red", label="Linear Fit", linewidth=2)
    plt.plot(x_vals[x_col], y_poly, color="blue", label="Polynomial Fit (Degree 2)", linewidth=2)
    plt.title("Measured Power vs. Model Predictions")
    plt.xlabel(x_col)
    plt.ylabel("Power (W)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def display_model_errors(model_df: pd.DataFrame) -> None:
    display_df = model_df.copy()

    display_df["error_linear"] = display_df["error_linear"].map(lambda x: f"{x:.2f}%")
    display_df["error_poly"] = display_df["error_poly"].map(lambda x: f"{x:.2f}%")

    for col in ["cores", "avg_power", "predicted_linear", "predicted_poly"]:
        if col in display_df.columns:
            display_df[col] = display_df[col].round(2)

    display(display_df)

# === Model Persistence ===
def load_cpu_info(config: Dict) -> Dict:
    path = os.path.join(config["result_folder"], "config", config["server_name"], "hardware.json")
    with open(path, "r") as file:
        hardware = json.load(file)
        cpu = hardware.get("processor", [{}])[0]
        return {"cores": cpu.get("cores"), "model": cpu.get("model")}
    return {}

def load_gpu_info(config: Dict) -> Dict:
    path = os.path.join(config["result_folder"], "config", config["server_name"], "hardware.json")
    with open(path, "r") as f:
        hardware = json.load(f)
    gpus = hardware.get("graphics", []) or []
    by_model = {}
    for g in gpus:
        if not isinstance(g, dict):
            continue
        vendor = (g.get("vendor") or "Unknown").strip()
        model = (g.get("model") or "Unknown").strip()
        key = f"{vendor} {model}".strip()
        by_model[key] = by_model.get(key, 0) + 1
    return {"count": sum(by_model.values()), "by_model": by_model}


def save_energy_model(
    config: Dict,
    p_unit: float,                 # p_core for cpu OR p_load for gpu
    intercept: float,
    poly_coeffs: Optional[Tuple[float, float, float]] = None,
    mapping: Optional[Dict] = None,
    cpu_info: Optional[Dict] = None,
    gpu_info: Optional[Dict] = None,
) -> Dict:
    kind = config.get("model_kind", "cpu")
    out_folder = CPU_MODEL_FOLDER if kind == "cpu" else GPU_MODEL_FOLDER

    output_name = f"{config['model_output']}_{config['server_name']}.json"
    output_path = os.path.join(out_folder, output_name)

    model = {
        "model_kind": kind,
        "model_type": "linear_fixed_idle" if config.get("fix_idle") else "linear_free_fit",
        "linear_model": {
            "p_base": config["idle_power"],
            "p_unit": p_unit,               # cpu: W/core, gpu: W/(1.0 load)
            "fitted_intercept": intercept
        },
        "x_col": config["x_col"],           # "cores" or "gpu_load"
        "timestamp": datetime.now().isoformat(),
        "node_name": config["server_name"],
        "source_files": mapping or {},
        "cpu_info": cpu_info or {},
        "gpu_info": gpu_info or {},
    }

    if poly_coeffs is not None:
        a, b, c = poly_coeffs
        model["polynomial_model"] = {"a": a, "b": b, "c": c}

    os.makedirs(out_folder, exist_ok=True)
    with open(output_path, "w") as f:
        json.dump(model, f, indent=2)
    print(f"Saved model to {output_path}")
    return model


## Experiment Configuration

The CONFIG dictionary defines the core setup for running a CPU power modeling experiment.  
It controls where the data comes from, how the models are built, and where results are saved.

### Configuration Fields

| Key               | Type    | Description 
|------------------|---------|-------------
| result_folder  | str   | Root path to the experiment results. This is used to locate CSV files (under energy/) and system metadata (hardware.json under config/). 
| server_name    | str   | Logical name or hostname of the test server. Used to build subpaths to results and metadata folders. 
| idle_power     | float | Idle (baseline) power consumption in watts. Used as a reference point to calculate per-core dynamic power. 
| csv_base_name  | str   | Prefix for measurement CSV filenames (e.g., measurement_run0.csv, measurement_run1.csv, ...). 
| csv_extension  | str   | File extension for the measurement files. Typically .csv. 
| map_to         | str   | Logical label for mapping measurements to a parameter (e.g., "cores" to track how many CPU cores were active). Mostly used in logging and plots. 
| model_output   | str   | Base filename for the saved CPU power model. Final output will be stored in the cpu_models/ folder as model_output_server_name.json. 
| fix_idle       | bool  | If True, forces the model to use a fixed intercept at idle_power. If False, the model fits the intercept freely based on data. 


### Additional Behavior

- If start_index and end_index are not provided, they are automatically inferred from the number of CPU cores (based on hardware.json).
- Models are saved as JSON files and include timestamps, node info, and fitted parameters.
- Filenames for data are automatically constructed using {csv_base_name}{index}{csv_extension}

In [ ]:
CONFIG = build_config()
run_mapping = generate_run_mapping(CONFIG)

# Load Stress Run Data

Each CSV file represents a stress run using a known number of CPU cores.  
We compute the average power and derive the per-core contribution using:

$$
P_{\text{core}} = \frac{\bar{P}_{\text{measured}} - P_{\text{base}}}{\text{\# active cores}}
$$

In [ ]:
# csv_folder = get_csv_folder(CONFIG)
# model_df = load_power_data(csv_folder, run_mapping, CONFIG["idle_power"])


csv_folder = get_csv_folder(CONFIG)
model_df = load_power_data(
    csv_folder,
    run_mapping,
    CONFIG["idle_power"],
    x_col=CONFIG["x_col"],
    model_kind=CONFIG["model_kind"]
)

# Model Construction

We fit two models to understand how CPU usage affects total power consumption:

- **Linear Regression**: Assumes each core adds a constant amount of power.
- **Polynomial Regression (Degree 2)**: Allows for non-linear effects like saturation or thermal throttling.

The model is used in this generalized prediction formula:

$$
P_{\text{server}} = P_{\text{base}} + \sum_{i=1}^{n} \lambda_i \cdot P_{\text{core}, i} + \sum_{j=1}^{k} \lambda_j \cdot P_{\text{nic}, j}
$$

Where:

- $P_{\text{server}}$: Total predicted server power (watts)  
- $P_{\text{base}}$: Idle power consumption of the server (measured)  
- $n$: Number of active CPU cores  
- $\lambda_i$: Load factor (0–1) of CPU core $i$  
- $P_{\text{core}, i}$: Power used by CPU core $i$ at full load  
- $k$: Number of active NICs  
- $\lambda_j$: Activity factor (0–1) of NIC $j$ (simplified as 1 if enabled, 0 if disabled)  
- $P_{\text{nic}, j}$: Power draw of NIC $j$ at full operation  

**Note:**  
- NIC power values are currently based on fixed, hardcoded estimates. These are placeholder values and do not reflect dynamic NIC behavior yet.  
- CPU power is the dominant component in most server workloads, so this approximation still yields useful insights.  
- By using a measured idle power ($P_{\text{base}}$) for each node, we account for node-specific baseline consumption (e.g. mainboard, PSU, background services), which helps normalize comparisons across different server hardware.

In [ ]:
reg_linear, reg_poly, poly = fit_models(
    model_df,
    CONFIG["idle_power"],
    x_col=CONFIG["x_col"],
    fix_idle=CONFIG["fix_idle"]
)

# Save Model

Export the linear model (with or without fixed idle intercept) as a reusable .json file.  
This can later be used for power estimation.

In [ ]:
# After fitting
p_unit = reg_linear.coef_[0]
intercept = CONFIG["idle_power"] if CONFIG["fix_idle"] else reg_linear.intercept_
poly_coeffs = (reg_poly.coef_[2], reg_poly.coef_[1], reg_poly.intercept_)

cpu_info = load_cpu_info(CONFIG) if CONFIG["model_kind"] == "cpu" else load_cpu_info(CONFIG)  # optional to always include
gpu_info = load_gpu_info(CONFIG) if CONFIG["model_kind"] == "gpu" else load_gpu_info(CONFIG)  # optional to always include

energy_model = save_energy_model(
    config=CONFIG,
    p_unit=p_unit,
    intercept=intercept,
    poly_coeffs=poly_coeffs,
    mapping=run_mapping,
    cpu_info=cpu_info,
    gpu_info=gpu_info
)

# Model Fit Visualization

We compare the measured power values to the predictions from:

- **Linear Regression**  
- **Polynomial Regression (Degree 2)**

This plot helps assess how well each model captures the real trend in power usage as more CPU cores are utilized.

We plot:

- Measured values as scatter points  
- Linear model as a red line  
- Polynomial model as a blue curve

In [ ]:
plot_model_fit(model_df, reg_linear, reg_poly, poly, x_col=CONFIG["x_col"])

## Prediction Accuracy: Linear vs. Polynomial

Below is a comparison table showing the actual power consumption values (real_power), the predicted values from both models, and their corresponding percentage errors.
This helps evaluate how well each model fits the data.


In [ ]:
display_model_errors(model_df)